In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
sys.path.append("../")
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from PIL import Image
from src.train_pm import Dataset, double_conv, LeUNet
import cv2
from matplotlib.pyplot import imshow
from src.preprocessing.trans_func import *

In [26]:
import torchvision.models as models

In [27]:
from src.train_pm import Dataset, double_conv, LeUNet, StandardNet, EnsembleNet, EPAPLN, ResNetUNet, EnsembleLeUNet

In [4]:
dfpm = pd.read_csv('../Ghaziabad_Images_wPM25.csv')

In [11]:
def ts_format(a):
    a = a[:-6]
    a = a.replace('-','')
    a = a.replace(':','')
    a = a.replace(' ','_')
    return a

In [13]:
dfpm['timestamp'] = dfpm['timestamp'].apply(ts_format)

In [19]:
dfpm = dfpm.set_index(['timestamp'])

In [15]:
import os

In [16]:
img_list = os.listdir('../Ghaziabad_Images/')

In [20]:
df = pd.DataFrame(columns=['filename','ppm'])

In [24]:
df.to_csv('../pilot_data.csv', index=False)

In [38]:
data = pd.read_csv('../all_data.csv')
data_train = data.sample(frac=0.8,random_state=17)
data_val = data.loc[~data.index.isin(data_train.index)]
files_train = list(data_train['filename'])
files_val = list(data_val['filename'])
ppm_train = list(data_train['ppm'])
ppm_val = list(data_val['ppm'])
ids_train = [i for i in range(len(files_train))]
ids_val = [i for i in range(len(files_val))]
# data = None
# data_train = None
# data_val = None
files = list(data['filename'])
ppm = list(data['ppm'])
ids = [i for i in range(len(files))]

In [47]:
model = LeUNet()
model = torch.nn.DataParallel(model).cuda()
model.load_state_dict(torch.load("../src/model_haze_all.pth"),strict=False) # on GPU

_IncompatibleKeys(missing_keys=['module.features.0.weight', 'module.features.0.bias', 'module.features.3.weight', 'module.features.3.bias', 'module.estimator.0.weight', 'module.estimator.0.bias', 'module.estimator.2.weight', 'module.estimator.2.bias', 'module.estimator.4.weight', 'module.estimator.4.bias'], unexpected_keys=[])

In [40]:
def freeze_weights_leunet(model):
    model.module.dconv_down1[0].weight.requires_grad, model.module.dconv_down1[0].bias.requires_grad = False, False
    model.module.dconv_down2[0].weight.requires_grad, model.module.dconv_down2[0].bias.requires_grad = False, False
    model.module.dconv_down3[0].weight.requires_grad, model.module.dconv_down3[0].bias.requires_grad = False, False
    model.module.dconv_down4[0].weight.requires_grad, model.module.dconv_down4[0].bias.requires_grad = False, False
    model.module.dconv_up1[0].weight.requires_grad, model.module.dconv_up1[0].bias.requires_grad = False, False
    model.module.dconv_up2[0].weight.requires_grad, model.module.dconv_up2[0].bias.requires_grad = False, False
    model.module.dconv_up3[0].weight.requires_grad, model.module.dconv_up3[0].bias.requires_grad = False, False
    model.module.dconv_down1[2].weight.requires_grad, model.module.dconv_down1[2].bias.requires_grad = False, False
    model.module.dconv_down2[2].weight.requires_grad, model.module.dconv_down2[2].bias.requires_grad = False, False
    model.module.dconv_down3[2].weight.requires_grad, model.module.dconv_down3[2].bias.requires_grad = False, False
    model.module.dconv_down4[2].weight.requires_grad, model.module.dconv_down4[2].bias.requires_grad = False, False
    model.module.dconv_up1[2].weight.requires_grad, model.module.dconv_up1[2].bias.requires_grad = False, False
    model.module.dconv_up2[2].weight.requires_grad, model.module.dconv_up2[2].bias.requires_grad = False, False
    model.module.dconv_up3[2].weight.requires_grad, model.module.dconv_up3[2].bias.requires_grad = False, False
    model.module.conv_last.weight.requires_grad, model.module.conv_last.bias.requires_grad = False, False
    return model

In [41]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr=1e-4)

In [42]:
criterion = nn.MSELoss().cuda()

In [43]:
train_dataset = Dataset(ids_train, files_train, ppm_train, transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor(),transforms.Normalize(mean=[0.5231, 0.5180, 0.5115],std=[0.2014, 0.2018, 0.2100]),])) # normalize
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=12)
val_dataset = Dataset(ids_val, files_val, ppm_val, transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor(),transforms.Normalize(mean=[0.5231, 0.5180, 0.5115],std=[0.2014, 0.2018, 0.2100]),]))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=12)

In [44]:
def train(train_loader,model,criterion,optimizer):
    model.train()
    total_loss = 0.0
    epoch_samples = 0
    for x, y in train_loader:
        epoch_samples += x.size(0)
        y = y.float()
        x = x.cuda(non_blocking=True)
        y = y.cuda(non_blocking=True)

        x_var = torch.autograd.Variable(x)
        y_var = torch.autograd.Variable(y)

        yhat = model(x_var)
        loss = criterion(yhat.squeeze(),y_var)
        total_loss += loss.data.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return (total_loss/epoch_samples)


def val(val_loader,model,criterion):
    model.eval()
    total_loss = 0.0
    epoch_samples = 0
    #with torch.no_grad():
    for x, y in val_loader:
        epoch_samples += x.size(0)
        y = y.float()
        x = x.cuda(non_blocking=True)
        y = y.cuda(non_blocking=True)

        x_var = torch.autograd.Variable(x)
        y_var = torch.autograd.Variable(y)

        yhat = model(x_var)
        loss = criterion(yhat.squeeze(),y_var)
        total_loss += loss.data.item()

    return (total_loss/epoch_samples)

In [48]:
model = freeze_weights_leunet(model)

In [49]:
best_loss = 1e5
for epoch in range(500):
    train_loss = train(train_loader,model,criterion,optimizer)
    val_loss = val(val_loader,model,criterion)
    print('Epoch: %d, MSE train set: %.8f' % (epoch+1, train_loss))
    print('Epoch: %d, MSE val set: %.8f\n' % (epoch+1, val_loss))
    if val_loss < best_loss:
        torch.save(model.state_dict(),'modelfrozen_pm_all.pth')
        best_loss = val_loss

Epoch: 1, MSE train set: 718.68777649
Epoch: 1, MSE val set: 812.71084475



KeyboardInterrupt: 

In [2]:
df_annot = pd.read_csv('/scratch/ab9738/pollution_img/code/Annotations.csv')

In [6]:
df = df_annot.drop(['timestamp'], axis=1)

In [25]:
df.to_csv('/scratch/ab9738/pollution_img/code/annotations.csv', index=False)

In [18]:
dff = df.copy()

In [22]:
for i in range(len(df)):
    df.loc[i] = df.loc[i][0][:-4], df.loc[i][1]

In [24]:
df

,filename,closest_value
0,20220120_190949___28.552655_77.105907___[org],189.82
1,20220120_191219___28.550738_77.120168___[org],189.82
2,20220120_191326___28.553902_77.123568___[org],189.82
3,20220120_191443___28.55705_77.132781___[org],189.82
4,20220120_191528___28.561069_77.133684___[org],189.82
...,...,...
377,20220122_182803___28.47207_77.487583___[org],80.00
378,20220122_182839___28.474239_77.485458___[org],80.00
379,20220122_182847___28.474239_77.485458___[org],80.00
380,20220122_182904___28.475996_77.483764___[org],80.00
